In [3]:
import glob
import pandas as pd
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import re
from DataProcessing import *
from DataImporting import sequence_list
from venditti_data_importer import venditti_assignment
peak_list, residue_list = main_data_processing()

acceptance_threshold = .20
should_print_unassigned_lists = False
should_print_peak_data = True
should_export_to_excel = True
should_take_best_10 = False


# if you want to analyze multiple test conditions, add them here.
for random_index in range(1):
    # DATA INGESTION
    time_taken_list = list()
    final_energy_list = list()
    filename_list = list()
    count_of_assignments = 0

    # glob.glob returns a list of paths that match filename NAME.
    # add all files for each test condition

    NAME = ["Slurm Trials/slurm-4102660_{}.out".format(str(x)) for x in range(1, 17)] + \
           ["Slurm Trials/slurm-4109433_{}.out".format(str(x)) for x in range(17, 201)] + \
           ["Slurm Trials/slurm-9199622_{}.out".format(str(x)) for x in range(1, 81)]

    # NAME = ["40:30.out", "40:30_2.out"]



    set_iterations, initial_temperature, exponent, eif = None, None, None, None
    time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy = \
        None, None, None, None, None, None, None

    initial_data = pd.DataFrame(columns=['set_iterations', 'time_taken', 'completed_iterations', 'final_temp',
                                         'og_index_list', 'og_energy', 'index_list', 'final_energy'])

    for filename_str in NAME:
        # imports each file
        # filename = [i[0] for i in glob.glob(filename_str)]
        file = open(filename_str, 'r')
        filename_list.append(filename_str)

        for line in file.readlines():
            if "Done" in line:
                count_of_assignments += 1
                df_temp = pd.DataFrame([[set_iterations, time_taken, completed_iterations,
                                         final_temp, og_index_list, og_energy,
                                         index_list, final_energy]],
                                       columns=['set_iterations', 'time_taken', 'completed_iterations',
                                                'final_temp', 'og_index_list', 'og_energy',
                                                'index_list', 'final_energy'])
                initial_data = initial_data.append(df_temp, ignore_index=True)

                set_iterations, initial_temperature, exponent, eif = None, None, None, None
                time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy = \
                    None, None, None, None, None, None, None

            elif 'exponent:' in line:
                set_iterations = int(re.search(r" \d* ", line)[0][1:-1])
            elif 'time taken (sec)' in line:
                time_taken = float(re.search(r"\d*\.\d*", line)[0])
                time_taken_list.append(time_taken)
            elif 'number of iterations' in line:
                completed_iterations = int(re.search(r"[0-9].*", line)[0])
            elif 'final temperature' in line:
                final_temp = float(re.search(r"\d*\.\d*", line)[0])
                final_energy_list.append(final_temp)
            elif 'og index list:' in line:
                og_index_list = list()
                [og_index_list.append(int(x)) for x in line[line.index('[') + 1:line.index(']')].split(', ')]
            elif 'og index list energy:' in line:
                og_energy = float(re.search(r"\d*\.\d*", line)[0])
            elif 'index list:' in line:
                index_list = list()
                [index_list.append(int(x)) for x in line[line.index('[') + 1:line.index(']')].split(', ')]
            elif 'index list energy:' in line:
                final_energy = float(re.search(r"\d*\.\d*", line)[0])

    print(initial_data.head())
        # print(time_taken, completed_iterations, final_temp, og_index_list, og_energy, index_list, final_energy)

  set_iterations    time_taken completed_iterations final_temp  \
0      100000000  11913.965294            100000000       None   
1      100000000  10211.317212            100000000       None   
2      100000000   8680.950425            100000000       None   
3      100000000   8079.023462            100000000       None   
4      100000000   9645.769741            100000000       None   

                                       og_index_list     og_energy  \
0  [385, 226, 347, 348, 64, 232, 430, 376, 236, 1...  1.187004e+08   
1  [259, 97, 443, 118, 83, 216, 417, 108, 416, 38...  9.992624e+07   
2  [192, 296, 106, 425, 122, 200, 262, 207, 24, 4...  9.499531e+07   
3  [23, 369, 264, 40, 356, 345, 69, 341, 395, 167...  9.733047e+07   
4  [74, 363, 175, 91, 51, 463, 6, 325, 137, 327, ...  9.454681e+07   

                                          index_list  final_energy  
0  [131, 29, 257, 216, 93, 287, 402, 467, 206, 44...  1.750849e+06  
1  [234, 205, 257, 216, 45, 405, 349, 199, 2

In [5]:
initial_data = initial_data.sort_values(by='final_energy')

In [6]:
print(initial_data.iloc[0:10])

     set_iterations     time_taken completed_iterations final_temp  \
3303     5000000000  431399.647103           4787511967    191.581   
3301     5000000000  427560.215611           5000000000    191.581   
3335     5000000000  372451.363364           5000000000    191.581   
3349     5000000000  391068.616322           5000000000    191.581   
3347     5000000000  390870.787135           5000000000    191.581   
3288     5000000000  333553.390344           5000000000    191.581   
3363     5000000000  366911.160029           5000000000    191.581   
3345     5000000000  383293.910890           5000000000    191.581   
3291     5000000000  372435.033140           5000000000    191.581   
3330     5000000000  359109.094401           5000000000    191.581   

                                          og_index_list     og_energy  \
3303  [393, 450, 429, 445, 248, 124, 179, 185, 90, 7...  9.101356e+07   
3301  [159, 152, 41, 213, 8, 341, 363, 345, 267, 94,...  1.195123e+08   
3335  [215

In [24]:
for j, index_list in enumerate(best_list):
    data = pd.DataFrame(columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', ])
    for i, pai in enumerate(index_list):
        residue_number = i + 1
        amino_acid = sequence_list[i]
    
        peak = peak_list[pai]
        assert pai == peak.get_data('peakNumber')
        n_shift = peak.get_data('TROSYNShift')
        h_shift = peak.get_data('TROSYHShift')
        ca_shift = peak.get_data('CAShift')
        ca_prime_shift = peak.get_data('CAPrimeShift')
        cb_shift = peak.get_data('CBShift')
        cb_prime_shift = peak.get_data('CBPrimeShift')
    
    
        temp_df = pd.DataFrame([[residue_number, amino_acid,
                                 n_shift, h_shift, ca_shift, ca_prime_shift, cb_shift, cb_prime_shift]],
                               
                               columns=['residue number', 'amino acid', 
                                        'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', ])
        data = data.append(temp_df, ignore_index=True)
    print(data.head())
    data.to_csv('/Volumes/Transcend/Fawzi_pycharm_project/data_{}.csv'.format('40:30'))

  residue number amino acid        N      H      CA CAPrime      CB CBPrime
0              1          T  122.329  8.622   60.33  49.504    None  63.778
1              2          P  124.950  8.669    None  59.305    None  57.487
2              3          K  118.145  7.480  58.599  65.716  57.822    None
3              4          D  122.497  7.587    None  58.724    None  59.062
4              5          D  122.893  8.752   54.95  56.258  53.153  54.189


In [16]:
print(initial_data['final_energy'][3303])
index_list = initial_data['index_list'][3303]
print(eval_energy(index_list, peak_list, residue_list, energy_if_false)[0])
print(len(residue_list))

1585506.9112085525
829095.8058876285
474


In [35]:

# for j, index_list in enumerate([index_list]):

index_list = initial_data['index_list'][3303]
info_list = eval_energy(index_list, peak_list, residue_list, energy_if_false)[1]

data = pd.DataFrame(columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', 
                             'CACAPrime', 'CBCBPrime', 'CACBExp', 'CACBExpPrime', 'NOESY', 'NOESYSubWeights', 'NOESYDelH'])
for i, pai in enumerate(index_list):
    residue_number = i + 1
    amino_acid = sequence_list[i]

    peak = peak_list[pai]
    assert pai == peak.get_data('peakNumber')
    n_shift = peak.get_data('TROSYNShift')
    h_shift = peak.get_data('TROSYHShift')
    ca_shift = peak.get_data('CAShift')
    ca_prime_shift = peak.get_data('CAPrimeShift')
    cb_shift = peak.get_data('CBShift')
    cb_prime_shift = peak.get_data('CBPrimeShift')

    ca_ca_prime, cb_cb_prime, ca_cb_exp, = energy_if_false, energy_if_false, energy_if_false
    ca_cb_exp_prime, noesy_energy = energy_if_false, energy_if_false
    noesy_subweights, noesy_del_h = [], []
    
    for k, line in enumerate(info_list): 
        if len(line) == 2: 
            if line[0] == i: 
                assert line[1] == pai
                for l in range(1,6):
                    if len(info_list[k + l]) == 2: 
                        break
                    elif 'CACAPrime' == info_list[k + l][0]: 
                        ca_ca_prime = info_list[k + l][1]
                    elif 'CBCBPrime' == info_list[k + l][0]: 
                        cb_cb_prime = info_list[k + l][1]
                    elif 'CACBExp' == info_list[k + l][0]: 
                        ca_cb_exp = info_list[k + l][1]
                    elif 'CACBExpPrime' == info_list[k + l][0]: 
                        ca_cb_exp_prime = info_list[k + l][1]
                    elif 'NOESY' == info_list[k + l][0]: 
                        noesy_energy = info_list[k + l][1]
                        noesy_subweights = info_list[k + l][2]
                        noesy_del_h = info_list[k + l][3]
                        
    temp_df = pd.DataFrame([[residue_number, amino_acid, n_shift, h_shift, ca_shift, ca_prime_shift, cb_shift, cb_prime_shift,
                             ca_ca_prime, cb_cb_prime, ca_cb_exp, ca_cb_exp_prime, noesy_energy, noesy_subweights, noesy_del_h]],
                           columns=['residue number', 'amino acid', 'N', 'H', 'CA', 'CAPrime', 'CB', 'CBPrime', 
                                    'CACAPrime', 'CBCBPrime', 'CACBExp', 'CACBExpPrime', 'NOESY', 'NOESYSubWeights', 'NOESYDelH'])
    data = data.append(temp_df, ignore_index=True)
print(data.head())
data.to_csv('/Volumes/Transcend/Fawzi_pycharm_project/data_{}.csv'.format('sources_of_energy'))

  residue number amino acid        N      H      CA CAPrime      CB CBPrime  \
0              1          T  123.833  8.324  63.609    None    None  54.871   
1              2          P  111.425  7.573  63.643   63.66    None   58.07   
2              3          K  118.145   7.48  58.599  65.716  57.822    None   
3              4          D  122.497  7.587    None  58.724    None  59.062   
4              5          D  121.518  7.953    None  52.002    None  67.677   

    CACAPrime   CBCBPrime      CACBExp  CACBExpPrime       NOESY  \
0    0.340683  300.000000     0.100000    300.000000  300.000000   
1  287.749150  300.000000     0.100000     24.923548  300.000000   
2    1.325135  102.035136  1522.139938     15.273683    0.510385   
3  300.000000  300.000000     0.100000   2269.465847    0.588254   
4  300.000000  300.000000     0.100000   1985.674149    0.635669   

                                     NOESYSubWeights  \
0                                                 []   
1   

In [32]:
ls = eval_energy(index_list, peak_list, residue_list, energy_if_false)[1][0:150]
for i in ls: 
    print(i)

(0, 234)
('CACAPrime', 0.3406829118599718, -0.05099999999999483)
('CACBExp', 0.1, 2.0189999999999984, 0)
(1, 205)
('CACAPrime', 287.74914983999815, -2.0729999999999933)
('CACBExp', 0.1, 0.11299999999999955, 0)
('CACBExpPrime', 24.923547584994264, 2.069999999999993, 11.68)
(2, 257)
('CACAPrime', 1.3251352746394423, -0.125)
('CBCBPrime', 102.03513599999917, -1.2399999999999949)
('CACBExp', 1522.1399384338479, 2.198999999999998, 25.252000000000002)
('CACBExpPrime', 15.2736832080237, 2.185999999999993, 0)
('NOESY', 0.5103848139139651, [0.08264671879734088, 0.24639999999999998, 0.044000000000000004, 0.031112698372208092, 0.044000000000000004, 0.031112698372208092, 0.031112698372208092], [0.11299999999999955, 0.012000000000000455, 0.0, 0.038000000000000256, 0.06600000000000072, 0.9130000000000003, 0.13600000000000012])
(3, 216)
('CACBExp', 0.1, 0, 0)
('CACBExpPrime', 2269.465846557686, 2.323999999999998, 26.491999999999997)
('NOESY', 0.5882541897601063, [0.0731991153161525, 0.222101958568581

In [5]:
print('hi')

hi


In [11]:
for i in peak_list:
    print(i.get_data('HNCAHShift'))

5.774
5.264
6.311
6.407
6.302
6.312
7.019
7.154
6.969
6.668
6.884
7.083
7.12
7.262
7.518
7.483
7.443
7.551
7.492
7.402
7.39
7.368
7.43
7.481
7.407
7.405
7.412
7.318
7.048
7.105
6.923
6.832
7.01
6.954
None
7.053
6.789
6.645
7.646
7.68
7.637
7.701
7.947
8.019
8.046
8.208
8.475
8.74
8.899
8.974
9.156
9.26
8.834
9.134
8.789
8.48
9.906
9.297
9.349
9.51
9.422
9.037
8.712
8.412
None
8.741
8.857
8.916
8.858
8.815
8.701
8.568
8.628
8.494
8.589
8.656
8.71
8.693
9.077
9.401
None
9.552
9.421
9.408
9.18
9.048
None
8.96
8.976
8.884
8.858
8.844
None
8.767
9.031
8.226
8.398
8.207
8.266
7.992
7.858
7.91
7.814
8.57
8.543
8.404
9.188
9.27
None
8.952
None
8.811
None
8.624
8.62
8.664
8.625
8.534
None
8.557
None
8.819
8.781
8.726
None
9.198
None
9.208
9.123
9.051
8.792
None
8.748
8.802
8.836
8.84
8.698
8.661
8.665
8.596
8.602
8.49
8.411
8.374
8.44
8.467
8.487
None
8.414
8.36
8.682
8.618
8.475
8.526
8.523
8.555
8.379
8.404
8.931
None
8.582
8.477
8.614
8.264
8.431
8.203
8.154
8.394
8.003
7.892
7.895
None
None